# Using spaCy efficiently

This section introduces you to customising the spaCy pipeline, that is, what spaCy does with text and how.

After reading this section, you should know how to:

 - process collections of texts efficiently 
 - examine and extend the spaCy pipeline

Let's start by importing the spaCy library and the displacy module for drawing dependency trees.

In [1]:
# Import the spaCy library and the displacy module
from spacy import displacy
import spacy

We then import a language model for English.

In [2]:
# Load a small language model for English and assign it to the variable 'nlp'
nlp = spacy.load('en_core_web_sm')

# Call the variable to examine the object
nlp

Before we start, let's examine the spaCy _Language_ object in greater detail.

The _Language_ object is a pipeline that applies a language model to the input data.

We can examine the components of a pipeline via the `pipeline` attribute of a _Language_ object.

In [3]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1206080e0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x11fc52f40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1205a9460>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1205a95e0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x120611140>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x120671a00>)]

This returns a list of Python _tuples_ that consist of component names and the actual components that perform different natural language processing tasks.

## Processing texts efficiently

When working with larger volumes of data, processing the data as efficiently as possible is desirable.

To illustrate the best practices of processing texts efficiently using spaCy, let's define a toy example that consists of a Python list with three example sentences from the Wikipedia article on Barack Obama.

In [4]:
# Define a list of example sentences
sents = ["On October 1, 2009, the Obama administration went ahead with a Bush administration program, increasing nuclear weapons production.", 
         "The 'Complex Modernization' initiative expanded two existing nuclear sites to produce new bomb parts.", 
         "The administration built new plutonium pits at the Los Alamos lab in New Mexico and expanded enriched uranium processing at the Y-12 facility in Oak Ridge, Tennessee."]

# Call the variable to examine output
sents

['On October 1, 2009, the Obama administration went ahead with a Bush administration program, increasing nuclear weapons production.',
 "The 'Complex Modernization' initiative expanded two existing nuclear sites to produce new bomb parts.",
 'The administration built new plutonium pits at the Los Alamos lab in New Mexico and expanded enriched uranium processing at the Y-12 facility in Oak Ridge, Tennessee.']

This returns a list containing three sentences.

spaCy _Language_ objects have a specific method, `pipe()`, for processing texts stored in a Python list. The `pipe()` method has been optimised for this purpose, processing texts in _batches_ rather than as individual items, which makes this method faster than a traditional `for` loop.

The `pipe()` method takes a _list_ as input and returns a Python _generator_ named `pipe`.

In [5]:
# Feed the list of sentences to the pipe() method
docs = nlp.pipe(sents)

# Call the variable to examine the output
docs

<generator object Language.pipe at 0x120da35f0>

Generators are Python objects that contain other objects. When called, a generator object will yield objects contained within. 

To retrieve all objects in a generator, we must cast the output into another object type, such as a list. You can think of the list as a data structure that is able to collect the generator output.

In [6]:
# Cast the pipe generator into a list
docs = list(docs)

# Call the variable to examine the output
docs

[On October 1, 2009, the Obama administration went ahead with a Bush administration program, increasing nuclear weapons production.,
 The 'Complex Modernization' initiative expanded two existing nuclear sites to produce new bomb parts.,
 The administration built new plutonium pits at the Los Alamos lab in New Mexico and expanded enriched uranium processing at the Y-12 facility in Oak Ridge, Tennessee.]

This gives us a list of spaCy _Doc_ objects for further processing.

## Merging noun phrases

As the [previous section](basic_nlp.ipynb) showed, tasks such as part-of-speech tagging and dependency parsing make predictions about individual _Tokens_ and their properties, such as part-of-speech tags or syntactic dependencies.

Occasionally, however, it may be more beneficial to operate with larger linguistic units, so such as noun phrases that consist of multiple _Tokens_.

spaCy provides access to noun phrases via the attribute `noun_chunks` of a _Doc_ object.

Let's print out the noun chunks in each _Doc_ object contained in the list `docs`.

In [7]:
# Define the first for-loop over the list 'docs'
# The variable 'doc' refers to items in the list
for doc in docs:
    
    # Loop over each noun chunk in the Doc object
    for noun_chunk in doc.noun_chunks:
        
        # Print noun chunk
        print(noun_chunk)

October
the Obama administration
a Bush administration program
nuclear weapons production
The 'Complex Modernization' initiative
two existing nuclear sites
new bomb parts
The administration
new plutonium pits
the Los Alamos lab
New Mexico
enriched uranium processing
the Y-12 facility
Oak Ridge
Tennessee


These two `for` loops return several noun phrases that consist of multiple _Tokens_.

For merging noun phrases into a single _Token_, spaCy provides a function named `merge_noun_tokens` that can be added to the pipeline stored in a _Language_ object using the `add_pipe` method.

In [8]:
# Add component that merges noun phrases into single Tokens
nlp.add_pipe('merge_noun_chunks')

<function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>

In this case, we do not need to reassign the _Language_ object under `nlp` to the same variable to update its contents. The `add_pipe` method adds the component to the _Language_ object automatically. 

We can verify that the component was added successfully by examining the `pipeline` attribute under the _Language_ object `nlp`. 

In [9]:
# List the pipeline components
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1206080e0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x11fc52f40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1205a9460>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1205a95e0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x120611140>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x120671a00>),
 ('merge_noun_chunks',
  <function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>)]

As you can see, the final tuple in the list consists of the `merge_noun_chunks` function.

To examine the consequences of adding this function to the pipeline, let's process the three example sentences again using the `pipe()` method of the _Language_ object `nlp`.

We overwrite the previous results stored under the same variable by assigning the output to the variable `docs`.

Note that we also cast the result into a list by wrapping the _Language_ object and the `pipe()` method into a `list()` function.

In [10]:
# Apply the Language object 'nlp' to the list of sentences under 'sents'
docs = list(nlp.pipe(sents))

# Call the variable to examine the output
docs

[On October 1, 2009, the Obama administration went ahead with a Bush administration program, increasing nuclear weapons production.,
 The 'Complex Modernization' initiative expanded two existing nuclear sites to produce new bomb parts.,
 The administration built new plutonium pits at the Los Alamos lab in New Mexico and expanded enriched uranium processing at the Y-12 facility in Oak Ridge, Tennessee.]

Superficially, everything remains the same: the list contains three _Doc_ objects. 

However, if we loop over the _Tokens_ in the first _Doc_ object in the list, which can be accessed using brackets at position zero, e.g. `[0]`, we can see that the noun phrases are now merged and tagged using the label `NOUN`.

In [11]:
# Loop over Tokens in the first Doc object in the list
for token in docs[0]:
    
    # Print out the Token and its part-of-speech tag
    print(token, token.pos_)

On ADP
October PROPN
1 NUM
, PUNCT
2009 NUM
, PUNCT
the Obama administration NOUN
went VERB
ahead ADV
with ADP
a Bush administration program NOUN
, PUNCT
increasing VERB
nuclear weapons production NOUN
. PUNCT


Tagging noun phrases using the label `NOUN` is a reasonable approximation, as noun phrases typically take on similar grammatical functions as nouns.

As rendering the syntactic parse using displacy shows, merging the noun phrases simplifies the parse tree.

In [12]:
displacy.render(docs[0], style='dep')

Although the noun phrases are now represented by single *Tokens*, the noun chunks are still available under the `noun_chunks` attribute of the *Doc* object.

As shown below, spaCy stores noun chunks as *Spans*, whose `start` attribute determines the index of the Token where the _Span_ starts, while the `end` attribute determines where the _Span_ has ended.

This information is useful, if the syntactic analysis reveals patterns that warrant a closer examination of the noun chunks and their structure.

In [13]:
# Loop over the noun chunks in the first Doc object [0] in the list 'docs'
for noun_chunk in docs[0].noun_chunks:
    
    # Print out noun chunk, its type, the Token where the chunks starts and where it ends
    print(noun_chunk, type(noun_chunk), noun_chunk.start, noun_chunk.end)

October <class 'spacy.tokens.span.Span'> 1 2
the Obama administration <class 'spacy.tokens.span.Span'> 6 7
a Bush administration program <class 'spacy.tokens.span.Span'> 10 11
nuclear weapons production <class 'spacy.tokens.span.Span'> 13 14


## Merging named entities

Named entities can be merged in the same way as noun phrases by providing `merge_entities` to the `add_pipe()` method of the *Language* object.

Let's start by removing the `merge_noun_chunks` function from the pipeline.

The `remove_pipe()` method can be used to remove a pipeline component.

In [14]:
# Remove the 'merge_noun_chunks' function from the pipeline under 'nlp'
nlp.remove_pipe('merge_noun_chunks')

# Process the original sentences again
docs = list(nlp.pipe(sents))

The method returns a tuple containing the name of the removed component (in this case, a function) and the component itself.

We can verify this by calling the `pipeline` attribute of the _Language_ object `nlp`.

In [15]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1206080e0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x11fc52f40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1205a9460>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1205a95e0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x120611140>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x120671a00>)]

Finally, let's add the `merge_entities` component to the pipeline under `nlp`.

In [16]:
# Add the 'merge_entities' function to the pipeline
nlp.add_pipe('merge_entities')

# Process the data again
docs = list(nlp.pipe(sents))

Let's examine the result by looping over the _Tokens_ in the third _Doc_ object.

In [21]:
# Loop over Tokens in the third Doc object in the list
for token in docs[2]:
    
    # Print out the Token and its part-of-speech tag
    print(token, token.pos_)

The DET
administration NOUN
built VERB
new ADJ
plutonium NOUN
pits NOUN
at ADP
the DET
Los Alamos PROPN
lab NOUN
in ADP
New Mexico PROPN
and CCONJ
expanded VERB
enriched ADJ
uranium NOUN
processing NOUN
at ADP
the DET
Y-12 PROPN
facility NOUN
in ADP
Oak Ridge PROPN
, PUNCT
Tennessee PROPN
. PUNCT


Named entities that consist of multiple *Tokens*, as exemplified by place names such as "Los Alamos" and "New Mexico", have been merged into single *Tokens*.

This section should have given you an idea of how spaCy can be modified and used effectively.

The [following section](evaluating_nlp.ipynb) introduces you to evaluating the performance of language models. 